# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [104]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [105]:
document_tree = ET.parse( './data/mondial_database_less.xml' )


In [106]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [107]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [154]:
document = ET.parse( './data/mondial_database.xml' )
type(document)

xml.etree.ElementTree.ElementTree

In [249]:

for country in document.iterfind('country'):
    mortality = country.find('infant_mortality')
    if mortality is not None:
        print country.find('name').text, float(mortality.text)


Albania 13.19
Greece 4.78
Macedonia 7.9
Serbia 6.16
Andorra 3.69
France 3.31
Spain 3.33
Austria 4.16
Czech Republic 2.63
Germany 3.46
Hungary 5.09
Italy 3.31
Liechtenstein 4.33
Slovakia 5.35
Slovenia 4.04
Switzerland 3.73
Belarus 3.64
Latvia 7.91
Lithuania 6.0
Poland 6.19
Ukraine 8.1
Russia 7.08
Belgium 4.18
Luxembourg 4.28
Netherlands 3.66
Bosnia and Herzegovina 5.84
Croatia 5.87
Bulgaria 15.08
Romania 10.16
Turkey 21.43
Denmark 4.1
Estonia 6.7
Faroe Islands 5.71
Finland 3.36
Norway 2.48
Sweden 2.6
Monaco 1.81
Gibraltar 6.29
Guernsey 3.47
Iceland 3.15
Ireland 3.74
San Marino 4.52
Jersey 3.86
Malta 3.59
Isle of Man 4.17
Moldova 12.93
Portugal 4.48
United Kingdom 4.44
Afghanistan 117.23
China 14.79
Iran 39.0
Pakistan 57.48
Tajikistan 35.03
Turkmenistan 38.13
Uzbekistan 19.84
Armenia 13.97
Georgia 16.68
Azerbaijan 26.67
Bahrain 9.68
Bangladesh 45.67
Myanmar 44.91
India 43.19
Bhutan 37.89
Brunei 10.48
Malaysia 13.69
Laos 54.53
Thailand 9.86
Cambodia 51.36
Vietnam 18.99
Kazakhstan 21.61
No

# 10 countries with the lowest infant mortality rates

In [255]:

mortality_dict  = {}
for country in document.iterfind('country'):
    mortality = country.find('infant_mortality')
    if mortality is not None:
        mortality_dict[country.find('name').text] = float(mortality.text)
mortality = pd.DataFrame(mortality_dict.items(), columns=['name', 'infant_mortality'])
mortality.sort_values(by = 'infant_mortality').head(10)

,name,infant_mortality
35,Monaco,1.81
210,Japan,2.13
73,Norway,2.48
66,Bermuda,2.48
78,Singapore,2.53
108,Sweden,2.60
57,Czech Republic,2.63
145,Hong Kong,2.73
54,Macao,3.13
188,Iceland,3.15


# 10 cities with the largest population

In [256]:

city_population = []
for city in document.findall('.//city'):
    name = city.find('name')
    if  name is None:
        name = np.nan
    else:
        name = name.text
    population = city.find('population[@year="2011"]') 
    if population is None:
        population = np.nan
    else:
        population = int(population.text)
        city_population.append([name, population])
        
df = pd.DataFrame(city_population, columns=['name', 'population'])
df.sort_values('population', ascending=False).head(10)


,name,population
529,Mumbai,12442373
554,Delhi,11034555
523,Bangalore,8443675
418,London,8250205
487,Tehran,8154051
505,Dhaka,7423137
558,Hyderabad,6731790
518,Ahmadabad,5577940
627,Luanda,5000000
542,Chennai,4646732


# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [277]:

ethnicgroup_pop = {}

for country in document.iterfind('country'):
    population_percentage = []
    
    for population in country.findall('population'):
        population_percentage.append((int(population.get('year')), int(population.text)))
    max_population_percentage = max(population_percentage)[1]
    
    for ethnicgroup in country.findall('ethnicgroup'):
        population_ethnic_group = (float(ethnicgroup.get('percentage')) * max_population_percentage / 100)
        ethnicgroup_pop[ethnicgroup.text] =  ethnicgroup_pop.setdefault(ethnicgroup.text, 0) + population_ethnic_group
# did not understand why adding default is giving the correct results. Check with mentor        
ethnic_population = pd.DataFrame.from_dict(ethnicgroup_pop, orient='index')
ethnic_population.sort_values([0],ascending=False).head(10)


,0
Han Chinese,"1,245,058,800.00"
Indo-Aryan,"871,815,583.44"
European,"494,872,219.72"
African,"318,325,120.37"
Dravidian,"302,713,744.25"
Mestizo,"157,734,354.94"
Bengali,"146,776,916.72"
Russian,"131,856,996.08"
Japanese,"126,534,212.00"
Malay,"121,993,550.37"


# name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [290]:
code_to_country_dict = {}
for country in document.iterfind('country'):
    code_to_country_dict[country.get('car_code')] = country.find('name').text

rivers = []
for river in document.iterfind('river'):
    for country in river.get('country').split():
        length = river.find('length')
        if length is None:
            length = np.nan
        else:
            length = float(length.text)
        rivers.append([river.find('name').text, length, code_to_country_dict[country]])
river_df = pd.DataFrame(rivers, columns=['name', 'length', 'country'])
longest_river = river_df.loc[river_df['length'].idxmax()]
longest_river.to_frame()

,298
name,Amazonas
length,"6,448.00"
country,Colombia


In [291]:
code_to_country_dict = {}
for country in document.iterfind('country'):
    code_to_country_dict[country.get('car_code')] = country.find('name').text

lakes = []
for lake in document.iterfind('lake'):
    for country in lake.get('country').split():
        area = lake.find('area')
        if area is None:
            area = np.nan
        else:
            area = float(area.text)
        lakes.append([lake.find('name').text, area, code_to_country_dict[country]])
lake_df = pd.DataFrame(lakes, columns=['name', 'area', 'country'])

longest_lake = lake_df.loc[lake_df['area'].idxmax()]
longest_lake.to_frame()

,68
name,Caspian Sea
area,"386,400.00"
country,Russia


In [292]:
code_to_country_dict = {}
for country in document.iterfind('country'):
    code_to_country_dict[country.get('car_code')] = country.find('name').text
    

airports = []
for airport in document.iterfind('airport'):
    for country in airport.get('country').split():
        elevation = airport.find('elevation')
        if elevation is None or elevation.text is None:
            continue
        else:
            elevation = float(elevation.text)
        airports.append([airport.find('name').text, elevation, code_to_country_dict[country]])
airport_df = pd.DataFrame(airports, columns=['name', 'elevation', 'country'])

highelevation_airport = airport_df.loc[airport_df['elevation'].idxmax()]
highelevation_airport.to_frame()


,80
name,El Alto Intl
elevation,"4,063.00"
country,Bolivia
